<a href="https://colab.research.google.com/github/Antonio-B85/challenge-Parte-2--TelecomX/blob/main/TelecomX_Parte2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.- Librerías Básicas

In [1]:
# importar librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import classification_report

import warnings

warnings.filterwarnings('ignore')

# Ajustar la configuración para mostrar todas las columnas
pd.set_option('display.max_columns', None)
# Ajustar la configuración para mostrar todas las filas
pd.set_option('display.max_rows', None)
# Ajustar la configuración para mostrar más caracteres en cada celda
pd.set_option('display.max_colwidth', None)


#2.- Carga de Datos

In [2]:
# conectar a Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Rutas Drive
path = '/content/drive/MyDrive/Data_Science/Challenges/TelecomX-2/TelecomX_Data_Estandarizado.csv'

datos = pd.read_csv(path)
datos_org = datos.copy()
datos.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,19.776667
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,18.080000
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,9.361667
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,41.261667
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,8.913333


#3.- Análisis Inicial de Datos

In [4]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [5]:
datos.shape

(7267, 22)

In [6]:
datos.columns

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly', 'Charges.Total',
       'Cuentas_Diarias'],
      dtype='object')

#4.- Transformación de Datos

##4.1.- Eliminar Columnas y Filas Irrelevantes

In [7]:
# Eliminar la columna 'customerID'
datos = datos.drop(columns=['customerID'])
# Mostrar el DataFrame resultante
datos.head(3)

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias
0,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,19.776667
1,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,18.080000
2,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,9.361667


In [8]:
# Datos de la columna Churn
datos['Churn'].unique().tolist()

['No', 'Yes', 'No Info']

In [9]:
# Eliminar Filas "No Info" columna "Churn"
datos = datos[datos['Churn'] != 'No Info']

In [10]:
# Valido solo datos "Yes" o "No" en la Columna Churn
datos['Churn'].unique().tolist()

['No', 'Yes']

In [11]:
# Shaper original (7267, 22)
datos.shape

(7043, 21)

In [12]:
# Restablecer los índices
datos.reset_index(drop=True, inplace=True)
datos.tail()

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias
7038,No,Female,0,No,No,13,Yes,No,DSL,Yes,No,No,Yes,No,No,One year,No,Mailed check,55.15,742.90,24.763333
7039,Yes,Male,0,Yes,No,22,Yes,Yes,Fiber optic,No,No,No,No,No,Yes,Month-to-month,Yes,Electronic check,85.10,1873.70,62.456667
7040,No,Male,0,No,No,2,Yes,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,50.30,92.75,3.091667
7041,No,Male,0,Yes,Yes,67,Yes,No,DSL,Yes,No,Yes,Yes,No,Yes,Two year,No,Mailed check,67.85,4627.65,154.255000
7042,No,Male,0,Yes,Yes,63,No,No phone service,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Electronic check,59.00,3707.60,123.586667


##4.2.- Tratar Columnas Categóricas Binarias


In [13]:
datos.describe()

,SeniorCitizen,tenure,Charges.Monthly,Charges.Total,Cuentas_Diarias
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2283.296277,76.109876
std,0.368612,24.559481,30.090047,2265.000260,75.500009
min,0.000000,0.000000,18.250000,18.800000,0.626667
25%,0.000000,9.000000,35.500000,402.225000,13.407500
50%,0.000000,29.000000,70.350000,1400.550000,46.685000
75%,0.000000,55.000000,89.850000,3786.600000,126.220000
max,1.000000,72.000000,118.750000,8684.800000,289.493333


In [14]:
datos.describe(include='O')

,Churn,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,2,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4
top,No,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
freq,5174,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365


In [15]:
#Ajuste de datos
datos['Churn'] = datos['Churn'].replace({'No': 0, 'Yes': 1})
datos['gender'] = datos['gender'].replace({'Female': 0, 'Male': 1})
datos['Partner'] = datos['Partner'].replace({'No': 0, 'Yes': 1})
datos['Dependents'] = datos['Dependents'].replace({'No': 0, 'Yes': 1})
datos['PhoneService'] = datos['PhoneService'].replace({'No': 0, 'Yes': 1})
datos['PaperlessBilling'] = datos['PaperlessBilling'].replace({'No': 0, 'Yes': 1})

In [16]:
datos.sample(4)

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias
1007,1,1,0,0,0,1,1,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,1,Electronic check,89.10,89.10,2.970000
2613,1,1,0,0,0,1,1,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,1,Electronic check,88.35,88.35,2.945000
924,1,1,0,1,0,63,1,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,1,Electronic check,103.40,6603.00,220.100000
2334,0,1,1,1,0,36,1,No,DSL,No,No,Yes,Yes,No,No,One year,0,Credit card (automatic),54.10,1992.85,66.428333


In [17]:
# Validando columnas trabajadas no aparecen con mas de un valor
datos.describe(include='O')

,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaymentMethod
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,3,3,3,3,3,3,3,3,3,4
top,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Electronic check
freq,3390,3096,3498,3088,3095,3473,2810,2785,3875,2365
